In [ ]:
import pickle
import cv2
import os
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from PIL import Image

from utils.driver_state import determine_drive_state_changed

## Calculate driver change

In [ ]:
# Load optical flow data
optical_flow_data = torch.load(
    "resources/optical-flow/optical_flow.pkl", weights_only=False
)
records = []

# Iterate over the optical flow data to prepare a list of records
for video_filename, frames in optical_flow_data.items():
    # Correct video name format if needed
    if video_filename == "video_00013.mp4":
        video_filename = "video_0013.mp4"

    for frame_info in frames:
        records.append(
            {
                "filename": video_filename,
                "frame": frame_info["frame"],
                "score": frame_info["score"],
            }
        )

# Create a DataFrame from the records
optical_flow_df = pd.DataFrame(records)

# Extract the video name from the filename
optical_flow_df["video"] = optical_flow_df["filename"].apply(
    lambda filename: filename.split(".")[0]
)

# Add driver state change information
video_groups = []
for video_id, video_group in optical_flow_df.groupby("video"):
    # Backfill the missing score values
    time_series_scores = video_group["score"].bfill().reset_index(drop=True)

    # Determine the driver state change based on the backfilled scores
    video_group["change_bkp4"] = determine_drive_state_changed(
        time_series_scores, n_bkps=4
    )

    video_groups.append(video_group)

# Concatenate all video groups into a single DataFrame
final_optical_flow_df = pd.concat(video_groups).reset_index(drop=True)

## Create submission

In [ ]:
# Load submission structure
submission_df = pd.read_csv("./submissions/results_driverstate_alltracks_bboxsizes.csv")
submission_df["frame"] = submission_df["ID"].apply(
    lambda id_str: int(id_str.split("_")[-1])
)
submission_df["video"] = submission_df["ID"].apply(
    lambda id_str: "_".join(id_str.split("_")[:2])
)

# Load annotations
with open("./resources/annotations_public.pkl", "rb") as annotation_file:
    annotations = pickle.load(annotation_file)

# Prepare data structure for merging
annotation_data = []
for video_id, video_annotations in annotations.items():
    for frame_id, frame_annotations in video_annotations.items():
        annotation_data.append(
            {
                "ID": f"{video_id}_{frame_id}",
                "video": video_id,
                "frame": frame_id,
            }
        )

# Create DataFrame from annotations
annotation_df = pd.DataFrame(annotation_data)

# Merge annotations with submission data
merged_df = pd.merge(
    left=annotation_df, right=submission_df, how="left", on=["video", "frame"]
)

# Forward-fill missing data for each video
filled_df = merged_df.groupby("video").apply(lambda group: group.ffill())
filled_df = filled_df.reset_index(drop=True)

# Prepare final submission data
final_submission_df = filled_df[["ID", "change_bkp4"]].reset_index(drop=True)
final_submission_df = final_submission_df.rename(
    {"change_bkp4": "Driver_State_Changed"}, axis=1
)

# Save final submission to CSV
final_submission_df.to_csv(
    "./submissions/results_driverstate_alltracks_opticalflow.csv", index=False
)